# MICCAI 2023 Tutorial - Part II:
## Analyze your computation outputs
This part is designed to analyze the results from the [Hands-On Part I](1-launch-application.ipynb) and answer several **reproducibility questions**, such as:
- Are the outputs of the BraTS pipeline repeatable from one **execution** to another ?
- Are they reproducible between **version** 1.8.1 and version 1.9.0 ?
- Could we reproduce the numerical results of the [published paper](https://hal.science/hal-04006057) ?

> **N.B.:** At the time you start this Notebook, all tutorial executions may not be over yet. In such case, please **run this Notebook twice**: 
>-  The 1st time to focus on the outputs from the `vip-team`;
>- The 2nd time to update the analyses with the tutorial outputs.

---
## Useful libraries

Install: *Uncomment & run the following cell if you are using Google Colab*

In [ ]:
# Remove the hashtag (#) only
#! pip install matplotlib nibabel numpy pandas ipywidgets parse scipy vip-client girder-client

Import

In [ ]:
# Builtins
import matplotlib.pyplot as plt
from hashlib import md5
from pathlib import *
# Third-party
import nibabel as nib
import numpy as np
import pandas as pd
from ipywidgets import interact
from parse import parse
from scipy.ndimage import gaussian_filter
from vip_client.classes import VipLoader

---
## Understand the pipeline outputs

The next section will help you understand the outputs of the BraTS pipeline.
These outputs will be downloaded from the VIP servers and visualized in 3D.

<img src="imgs/BraTS-Pipeline.png" alt="BraTS-Pipeline" height="150" title="Full Pipeline for Brain Tumor Segmentation"/>

*In the cell below you need to provide again your VIP API key.*

In [ ]:
# Paste your VIP API key between the quotes
VipLoader.init(api_key="VIP_API_KEY");

First, download the outputs from previous executions.
- Outputs from the `vip-team` (paper results).
- Outputs from the `miccai-team`s (including yours) if all executions are over.

In [ ]:
# Local & distant paths to the dataset
res_dir = Path("data")
vip_dir = PurePosixPath("/vip/MICCAI tutorial (group)/outputs")
# Use the client to download the data
VipLoader.download_dir(vip_dir, "data")

The following cell shows two kinds of BraTS outputs :
1. a registered **brain scan** with skull strip;
2. a **tumor mask** relative the registered brain scan.

In [ ]:
# Define a filename for each type of result
filenames = {
    "tumor": 'brainTumorMask_SRI.nii.gz',
    "brain": 'T1_to_SRI_brain.nii.gz'
}
# Get 1 tumor file and 1 brain scan
tumor_file = next(res_dir.rglob('brainTumorMask_SRI.nii.gz'))
brain_file = next(res_dir.rglob('T1_to_SRI_brain.nii.gz'))
# Display their path
print("\n".join([str(tumor_file), str(brain_file)]))

Extract data from the brain & tumor files and display the brain volumes slice by slice 

In [ ]:
# Extract brain & tumor data from the previous files
brain = nib.load(brain_file).get_fdata()
tumor = nib.load(tumor_file).get_fdata()
tumor[tumor==0] = np.nan

# Interactive method to display the 3D images slice by slice
@interact
def show_slices(z=(0,150)) -> None:
    # Axes
    _, (ax_brain, ax_tumor) = plt.subplots(1, 2, figsize=(10,5))
    # Display the brain
    ax_brain.set_title("Brain Scan")
    ax_brain.imshow(brain[:,:,z], cmap='bone', origin="lower")
    ax_brain.axis('off')
    # Display the brain with tumor
    ax_tumor.set_title("With Tumor Detection")
    ax_tumor.imshow(brain[:,:,z], cmap='bone', origin="lower")
    ax_tumor.imshow(tumor[:,:,z], origin="lower")
    ax_tumor.axis('off')
    plt.show()

---
## Compare the outputs

The next section will help you compare BraTS outputs:
- Across pipeline **executions** from the same team (*exec_1*, *exec_2*, ...)
- Across pipeline **versions** (*1.8.1*, *1.9.0*)
- Across execution **teams** (*vip-team*, *miccai-teams*)

### List all output files
Build a [Dataframe](https://www.tutorialspoint.com/python_pandas/python_pandas_dataframe.htm) containing all files with relative metadata

In [ ]:
# List the result files
all_files = [str(path) 
    for type in filenames
    for path in res_dir.rglob(filenames[type])
]

# The file paths contain useful metadata
metadata_format = str(res_dir/"{Team}/{Version}/{Execution}/{_}/{Subject}/{File}")

# Build the dataset
# method
def get_metadata_from_path(path: str) -> dict:
    """Function to extract metadata from a single file path """
    metadata = parse(metadata_format, path)
    if metadata is None: 
        return {}
    result = metadata.named
    result.update({"Path": path}) 
    return result
# command
data = pd.DataFrame([get_metadata_from_path(file) for file in all_files])
# drop missing examples
data.dropna(axis=0, inplace=True) 

# Simplify the execution names
# method
def map_names(group: pd.Series):
    """Function to rename all executions of a given group in the dataset"""
    executions = group.unique()
    execution_map = { executions[i]: "exec_%d" %(i+1) for i in range(len(executions)) }
    return group.map(execution_map)
# command
data["Execution"] = data.groupby(["Team", "Version", "Subject"], group_keys=False)["Execution"].apply(map_names)

# Simplify the file names 
data["File"] = data["File"].map({
    'brainTumorMask_SRI.nii.gz': "tumor",
    'T1_to_SRI_brain.nii.gz': "brain",
})

# Metadata = Team, Version... everything but 'Path'
metadata_keys = list(data.columns)
metadata_keys.remove("Path")
data.set_index(metadata_keys, inplace=True)

# Display the dataset
data.head()

*You should see a table with file paths, file names, subjects, executions, versions & teams.*

### Compute the checksums
The following cell computes and displays the MD5 sums of all output files.

In [ ]:
# Create a Series of checksums from the file paths
# method
def md5sum(file: str) -> str:
    """Computes the md5sum of `file` path."""
    with open(file, "rb") as fid:
        return md5(fid.read()).hexdigest()
# command
checksums = data["Path"].apply(md5sum)

# Compare teams & executions
checksums.unstack(["Team", "Execution"])

*This table can help you answer the reproducibility questions raised in the introduction.*

### Image Analysis
This section helps visualizing the differences between tumor segmentations.

First, load the Nifti files:

In [ ]:
# Pre-load all Nifti files
# method
def load(file: str) -> str:
    """Loads a .nii file from its path"""
    with open(file, "rb") as fid:
        return nib.load(file)
# command
images = data["Path"].apply(load)

# Display the result
images.head()

Finally, show the differences in the segmented tumors between two `TEAMS`, pipeline `VERSIONS` or `EXECUTIONS`.

*Please modify the `TEAMS`, `VERSIONS` and `EXECUTIONS` in the cell below to observe the differences interactively.*

In [ ]:
# USER'S INPUTS -------------------------------------------------------

# Compare execution teams 
TEAM_A = "vip-team"
TEAM_B = "vip-team"
# TEAM_B = "miccai_team_1"

# Compare software versions 
VERSION_A = "v181"
VERSION_B = "v190"
# VERSION_B = "v181"

# Compare software executions 
EXECUTION_A = "exec_1"
EXECUTION_B = "exec_1"
# EXECUTION_B = "exec_2"

# INTERACTIVE DISPLAY --------------------------------------------------

# Function to display a tumor
def show_tumor(tumor: np.ndarray, brain: np.ndarray, ax: plt.Axes=plt):
    if brain is not None:
        ax.imshow(brain, cmap='bone', origin="lower")
    tumor[tumor==0] = np.nan
    ax.imshow(tumor, origin="lower")
    ax.axis('off')

def make_diff(tumor_a, tumor_b):
    values = np.unique(tumor_a)
    diff = np.zeros(np.shape(tumor_a))
    for val in values:
        diff[(tumor_a == val) ^ (tumor_b == val)] = val
    return diff

@interact
def show_diff(subject=list(images.index.get_level_values("Subject").unique()), sigma=(0, 1, 0.1), z=(0,150)):

    # Commmon Background: Registered brain image from output A
    brain = images[TEAM_A, VERSION_A, EXECUTION_A, subject, "brain"].get_fdata()[:,:,z]
    _, (ax_a, ax_diff, ax_b) = plt.subplots(1, 3, figsize=(15,5))

    # Image A : Tumor output A
    tumor_a = images[TEAM_A, VERSION_A, EXECUTION_A, subject, "tumor"].get_fdata()[:,:,z]
    show_tumor(tumor_a, brain, ax_a)
    ax_a.set_title("Output A")
    
    # Image B : Tumor output B
    tumor_b = images[TEAM_B, VERSION_B, EXECUTION_B, subject, "tumor"].get_fdata()[:,:,z]
    show_tumor(tumor_b, brain, ax_b)
    ax_b.set_title("Output B")

    # Difference
    tumor_diff = make_diff(tumor_a, tumor_b)
    tumor_diff = gaussian_filter(tumor_diff, sigma=sigma)
    show_tumor(tumor_diff, brain, ax_diff)
    ax_diff.set_title("Difference")

    plt.show()

*You should see 3 brain scans with segmented tumors.*
- *Use parameter `z` to navigate across the the Z-axis;*
- *Use parameter `sigma` to enhance the differences between outputs A and B.*

> **N.B.:** If some executions were not over when you started this Notebbok, you can **run again all cells from the top** to load and analyze all tutorial outputs.

---